In [219]:
import psycopg2
from zipfile import ZipFile
import pandas as pd
import os
from io import StringIO
import numpy as np
import shutil

zipped_folder=os.path.join('C:\\Users\\Munira\\Desktop\\demo-pipeline','zipped_raw_data')

unzipped_folder=os.path.join('C:\\Users\\Munira\\Desktop\\demo-pipeline','raw_data')

def connect_to_db(db_name,password):
    conn = psycopg2.connect(f"dbname={db_name} user=dataengineer host='mydb.cb7sygkkbh0u.us-east-2.rds.amazonaws.com' port=5432 password={password}")
    cur = conn.cursor()
    return conn,cur



## Modifying raw data files

In [60]:
os.mkdir(os.path.join(unzipped_folder,'CleanedData'))

In [90]:
os.path.join(unzipped_folder,'CleanedData').split('\\')[-1]

'CleanedData'

In [149]:
def clean_raw_data_files(original_filepath,cleaned_folder_path):

    df=pd.read_csv(original_filepath,encoding='cp1252')

    # Remove rows where all values are missing
    df=df.dropna(how='all')

    # Replacing empty strings for float columns with nans
    num_cols = [col for col in df.columns if df[col].dtype in ['float64','int64']]
    for col in num_cols:
        df[col]=df[col].fillna(np.nan)
        df.loc[df[col]=='',col]=np.nan

    # Replacing nulls with empty strings for text columns
    cols=[col for col in df.columns if df[col].dtype=='O']
    for col in cols:
        df[col]=df[col].fillna('')

    # Write into CSV file
    df.to_csv(os.path.join(cleaned_folder_path,os.path.split(original_filepath)[-1].replace('.txt','.csv')),index=False)


In [220]:
original_filepath=os.path.join(unzipped_folder,'DataFiles','2022Aug_Violation.txt')

In [221]:
cleaned_folder_path=os.path.join(unzipped_folder,'CleanedData')

In [222]:
clean_raw_data_files(original_filepath,cleaned_folder_path)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [223]:
df=pd.read_csv(original_filepath,encoding='cp1252')

# Remove rows where all values are missing
df=df.dropna(how='all')

# Replacing empty strings for float columns with nans
num_cols = [col for col in df.columns if df[col].dtype in ['float64','int64']]
for col in num_cols:
    df[col]=df[col].fillna(np.nan)
    df.loc[df[col]=='',col]=np.nan

# Replacing nulls with empty strings for text columns
cols=[col for col in df.columns if df[col].dtype=='O']
for col in cols:
    df[col]=df[col].fillna('')



In [234]:
df[df['SECTION_DESC']=='Failing to use seat belt while operating a CMV']

,UNIQUE_ID,INSP_DATE,DOT_NUMBER,VIOL_CODE,BASIC_DESC,OOS_INDICATOR,OOS_WEIGHT,SEVERITY_WEIGHT,TIME_WEIGHT,TOT_SEVERITY_WGHT,VIOL_VALUE,SECTION_DESC,GROUP_DESC,VIOL_UNIT
0,635384790.0,26-AUG-20,2582962.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
7,635384853.0,26-AUG-20,2104948.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
28,635385087.0,26-AUG-20,163666.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
33,635385123.0,26-AUG-20,1035752.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
60,635385492.0,26-AUG-20,709698.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055054,687955095.0,24-AUG-22,3348751.0,39216,Unsafe Driving,N,0.0,7.0,3.0,7.0,21.0,Failing to use seat belt while operating a CMV,Seat Belt,D
6055101,687955203.0,24-AUG-22,589183.0,39216,Unsafe Driving,N,0.0,7.0,3.0,7.0,21.0,Failing to use seat belt while operating a CMV,Seat Belt,D
6055102,687955212.0,24-AUG-22,1293502.0,39216,Unsafe Driving,N,0.0,7.0,3.0,7.0,21.0,Failing to use seat belt while operating a CMV,Seat Belt,D
6055203,687955545.0,24-AUG-22,2865468.0,39216,Unsafe Driving,N,0.0,7.0,3.0,7.0,21.0,Failing to use seat belt while operating a CMV,Seat Belt,D


In [230]:
df['GROUP_DESC'].unique()

array(['Seat Belt', 'Inspection Reports', 'Reflective Sheeting',
       'Lighting', 'Emergency Equipment', 'Phone Call',
       'Windshield/ Glass/ Markings', 'Brakes All Others',
       'Wheels Studs Clamps Etc.', 'Other Vehicle Defect',
       'Clearance Identification Lamps/Other', 'Cab Body Frame',
       'License-related: High', 'Incomplete/Wrong Log',
       'Brakes Out of Adjustment', 'Medical Certificate',
       'Brake Out Of Service', 'General Securement', 'Securement Device',
       'Improper Load Securement', 'Misc Violations', 'Tires',
       'Speeding 2', 'EOBR Related', 'Other Log/Form & Manner',
       'Exhaust Discharge', 'Hours', 'Failure to Prevent Movement',
       'Documentation - HM', 'Speeding 3', 'Fuel Systems',
       'Steering Mechanism', 'False Log', 'Load Securement - HM',
       'License-related: Medium', 'Tiedown', 'Tire vs. Load',
       'Hours Nominal', 'HM Other', 'General Driver Qualification',
       'Suspension', 'Other Driver Violations', 'Dangerous

In [227]:
df[df['GROUP_DESC']=='Failing to use seat belt while operating a CMV    ']

,UNIQUE_ID,INSP_DATE,DOT_NUMBER,VIOL_CODE,BASIC_DESC,OOS_INDICATOR,OOS_WEIGHT,SEVERITY_WEIGHT,TIME_WEIGHT,TOT_SEVERITY_WGHT,VIOL_VALUE,SECTION_DESC,GROUP_DESC,VIOL_UNIT


In [155]:
df['VIOL_UNIT'].unique()

array(['D', '1', '2', 'C', '4', '3', '5', '6'], dtype=object)

In [160]:
for col in [x for x in df.columns if df[x].dtype=='O']:
    print(f'col: {print(col)} len: {df[col].str.len().max()}')

INSP_DATE
col: None len: 9
VIOL_CODE
col: None len: 14
BASIC_DESC
col: None len: 36
OOS_INDICATOR
col: None len: 1
SECTION_DESC
col: None len: 162
GROUP_DESC
col: None len: 36
VIOL_UNIT
col: None len: 1


In [181]:
df['OOS_INDICATOR'].unique()

array(['N', 'Y'], dtype=object)

In [170]:
len('Failing to use seat belt while operating a CMV')

46

In [ ]:
\copy inspection FROM 'C:\\Users\Munira\Desktop\demo-pipeline\raw_data\CleanedData\2022Aug_Inspection.csv' WITH (FORMAT CSV,HEADER);


In [179]:
df[df['SECTION_DESC'].str.contains('ailing to use seat')]

,UNIQUE_ID,INSP_DATE,DOT_NUMBER,VIOL_CODE,BASIC_DESC,OOS_INDICATOR,OOS_WEIGHT,SEVERITY_WEIGHT,TIME_WEIGHT,TOT_SEVERITY_WGHT,VIOL_VALUE,SECTION_DESC,GROUP_DESC,VIOL_UNIT
0,635384790.0,26-AUG-20,2582962.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
7,635384853.0,26-AUG-20,2104948.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
28,635385087.0,26-AUG-20,163666.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
33,635385123.0,26-AUG-20,1035752.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
60,635385492.0,26-AUG-20,709698.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055054,687955095.0,24-AUG-22,3348751.0,39216,Unsafe Driving,N,0.0,7.0,3.0,7.0,21.0,Failing to use seat belt while operating a CMV,Seat Belt,D
6055101,687955203.0,24-AUG-22,589183.0,39216,Unsafe Driving,N,0.0,7.0,3.0,7.0,21.0,Failing to use seat belt while operating a CMV,Seat Belt,D
6055102,687955212.0,24-AUG-22,1293502.0,39216,Unsafe Driving,N,0.0,7.0,3.0,7.0,21.0,Failing to use seat belt while operating a CMV,Seat Belt,D
6055203,687955545.0,24-AUG-22,2865468.0,39216,Unsafe Driving,N,0.0,7.0,3.0,7.0,21.0,Failing to use seat belt while operating a CMV,Seat Belt,D


In [260]:
df.shape[1]

14

In [262]:
df.columns

Index(['UNIQUE_ID', 'INSP_DATE', 'DOT_NUMBER', 'VIOL_CODE', 'BASIC_DESC',
       'OOS_INDICATOR', 'OOS_WEIGHT', 'SEVERITY_WEIGHT', 'TIME_WEIGHT',
       'TOT_SEVERITY_WGHT', 'VIOL_VALUE', 'SECTION_DESC', 'GROUP_DESC',
       'VIOL_UNIT'],
      dtype='object')

In [263]:
df['VIOL_VALUE']

0           7.0
1           4.0
2           3.0
3           8.0
4           4.0
           ... 
6058000    12.0
6058001     3.0
6058002    30.0
6058003     6.0
6058004     6.0
Name: VIOL_VALUE, Length: 6058005, dtype: float64

In [ ]:
CREATE TABLE violation (
    Unique_ID FLOAT PRIMARY KEY,
    Insp_Date DATE NOT NULL,
    DOT_Number VARCHAR(50) NOT NULL,
    Viol_Code VARCHAR(50) NOT NULL,
    BASIC_Desc VARCHAR(50) ,
    OOS_Indicator CHAR(1),
    OOS_Weight REAL,
    Severity_Weight FLOAT,
    Time_Weight REAL,
    TOT_SEVERITY_WGHT FLOAT,
    Section_Desc VARCHAR(200) ,
    Group_Desc VARCHAR(200),
    Viol_Unit CHAR(2),
    V

In [261]:
df

,UNIQUE_ID,INSP_DATE,DOT_NUMBER,VIOL_CODE,BASIC_DESC,OOS_INDICATOR,OOS_WEIGHT,SEVERITY_WEIGHT,TIME_WEIGHT,TOT_SEVERITY_WGHT,VIOL_VALUE,SECTION_DESC,GROUP_DESC,VIOL_UNIT
0,635384790.0,26-AUG-20,2582962.0,39216,Unsafe Driving,N,0.0,7.0,1.0,7.0,7.0,Failing to use seat belt while operating a CMV,Seat Belt,D
1,635384790.0,26-AUG-20,2582962.0,3927A,Vehicle Maintenance,N,0.0,4.0,1.0,4.0,4.0,Driver failing to conduct pre-trip inspection,Inspection Reports,D
2,635384790.0,26-AUG-20,2582962.0,39311,Vehicle Maintenance,N,0.0,3.0,1.0,3.0,3.0,No or defective lighting devices or reflective...,Reflective Sheeting,1
3,635384790.0,26-AUG-20,2582962.0,39317B,Vehicle Maintenance,Y,2.0,6.0,1.0,8.0,8.0,No/defective towaway lamps on rear unit,Lighting,1
4,635384790.0,26-AUG-20,2582962.0,39617C,Vehicle Maintenance,N,0.0,4.0,1.0,4.0,4.0,Operating a CMV without proof of a periodic in...,Inspection Reports,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6058000,687977055.0,25-AUG-22,2626512.0,39353B,Vehicle Maintenance,N,0.0,4.0,3.0,4.0,12.0,CMV manufactured after 10/19/94 has an automat...,Brakes All Others,1
6058001,687977055.0,25-AUG-22,2626512.0,39360C,Vehicle Maintenance,N,0.0,1.0,3.0,1.0,3.0,Damaged or discolored windshield,Windshield/ Glass/ Markings,1
6058002,687977055.0,25-AUG-22,2626512.0,39375A1,Vehicle Maintenance,Y,2.0,8.0,3.0,10.0,30.0,Tire-ply or belt material exposed,Tires,1
6058003,687977055.0,25-AUG-22,2626512.0,3963A1,Vehicle Maintenance,N,0.0,2.0,3.0,2.0,6.0,Inspection repair and maintenance of parts & ...,Wheels Studs Clamps Etc.,1


In [180]:
df.iloc[0]

UNIQUE_ID                                               635384790.0
INSP_DATE                                                 26-AUG-20
DOT_NUMBER                                                2582962.0
VIOL_CODE                                                     39216
BASIC_DESC                                           Unsafe Driving
OOS_INDICATOR                                                     N
OOS_WEIGHT                                                      0.0
SEVERITY_WEIGHT                                                 7.0
TIME_WEIGHT                                                     1.0
TOT_SEVERITY_WGHT                                               7.0
VIOL_VALUE                                                      7.0
SECTION_DESC         Failing to use seat belt while operating a CMV
GROUP_DESC                                                Seat Belt
VIOL_UNIT                                                         D
Name: 0, dtype: object

In [130]:
df.columns

Index(['UNIQUE_ID', 'INSP_DATE', 'DOT_NUMBER', 'VIOL_CODE', 'BASIC_DESC',
       'OOS_INDICATOR', 'OOS_WEIGHT', 'SEVERITY_WEIGHT', 'TIME_WEIGHT',
       'TOT_SEVERITY_WGHT', 'VIOL_VALUE', 'SECTION_DESC', 'GROUP_DESC',
       'VIOL_UNIT'],
      dtype='object')

In [131]:
df.shape[1]

14

In [ ]:
# Write into CSV file
df.to_csv(os.path.join(cleaned_folder_path,os.path.split(original_filepath)[-1].replace('.txt','.csv')),index=False)


In [ ]:
import subprocess

## Copying into postgres

In [ ]:
\copy crash FROM 'C:\\Users\Munira\Desktop\demo-pipeline\raw_data\CleanedData\2022Aug_Crash.csv' WITH (FORMAT CSV, DELIMITER ',',HEADER);

\copy inspection FROM 'C:\\Users\Munira\Desktop\demo-pipeline\raw_data\CleanedData\2022Aug_Inspection.csv' WITH (FORMAT CSV, DELIMITER ',',HEADER);

In [ ]:
conn,cursor=connect_to_db('staging','aculocity')

cursor.execute(sql)
conn.commit()

In [203]:
unzipped_folder

'C:\\Users\\Munira\\Desktop\\demo-pipeline\\raw_data'

In [205]:
full_path=os.path.join(unzipped_folder,'CleanedData','2022Aug_Crash.csv')

In [258]:
sql= f"\copy crash FROM {full_path} WITH (FORMAT CSV,HEADER);"

In [259]:
sql

'\\copy crash FROM C:\\Users\\Munira\\Desktop\\demo-pipeline\\raw_data\\CleanedData\\2022Aug_Crash.csv WITH (FORMAT CSV,HEADER);'

In [237]:
os.getcwd()

'C:\\Users\\Munira\\Desktop\\demo-pipeline'

In [246]:
subprocess.Popen?

In [248]:
import subprocess

ssh = subprocess.Popen(sql.split(' '),
                        stdin =subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                       text=True,
                        shell=True)

In [249]:
ssh.stderr

<_io.TextIOWrapper name=9 encoding='cp1252'>

In [250]:
ssh.stdout

<_io.TextIOWrapper name=8 encoding='cp1252'>

In [253]:
ssh.communicate()

('',
 "'\\copy' is not recognized as an internal or external command,\noperable program or batch file.\n")